In [ ]:
# Cell 1: Installation
#!pip install -q --upgrade deep-translator pandas tqdm


In [ ]:
# Cell 2: Prepare Data Directory

# Sử dụng Google Colab để load nhanh dữ liệu

import os

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs('/content/data', exist_ok=True)

print("✅ Thư mục /content/data đã sẵn sàng.")
print("‼️ Vui lòng kéo và thả file 'spam.csv' của bạn vào thư mục này ở thanh bên trái.")

In [ ]:
# Cell 3: Setup Optimized Functions

# Sử dụng Google Colab để load nhanh dữ liệu

import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter
from tqdm import tqdm
from deep_translator import GoogleTranslator

# ==========================================================
# HÀM DỊCH BỀN BỈ (ROBUST BACK-TRANSLATION FUNCTION)
# Tự động thử lại khi gặp lỗi mạng hoặc bị giới hạn tốc độ.
# ==========================================================
def _back_translate_robust(sentence: str, source_lang: str = 'en', pivot_lang: str = 'vi', retries: int = 3, backoff_in_seconds: int = 5) -> str:
    """Thực hiện back-translation cho một câu với cơ chế thử lại."""
    original_sentence = sentence
    for i in range(retries):
        try:
            # Dịch xuôi (Anh -> Việt)
            translated = GoogleTranslator(source=source_lang, target=pivot_lang).translate(sentence)
            if not translated:
                continue # Nếu dịch ra rỗng, thử lại

            # Dịch ngược (Việt -> Anh)
            back_translated = GoogleTranslator(source=pivot_lang, target=source_lang).translate(translated)

            # Chỉ trả về câu mới nếu nó thực sự khác câu gốc
            if back_translated and back_translated.lower() != original_sentence.lower():
                return back_translated

            # Nếu không có gì thay đổi, chúng ta không cần câu này, trả về None để bỏ qua
            return None

        except Exception as e:
            # Nếu có lỗi, chờ một chút rồi thử lại
            print(f"⚠️ Lỗi dịch (lần {i+1}/{retries}): {e}. Đang thử lại sau {backoff_in_seconds} giây...")
            time.sleep(backoff_in_seconds)

    # Trả về None nếu thất bại sau tất cả các lần thử
    return None


# ==========================================================
# HÀM TĂNG CƯỜNG DỮ LIỆU SONG SONG (PARALLEL AUGMENTATION)
# ==========================================================
def augment_data_parallel(input_path: str, message_col: str, label_col: str, max_workers: int = 64) -> pd.DataFrame:
    """
    Tải dữ liệu, thực hiện augmentation song song và trả về DataFrame đã cân bằng.
    """
    print("🚀 Bắt đầu quá trình tăng cường dữ liệu...")
    df = pd.read_csv(input_path, encoding='latin1') # Dùng encoding latin1 phổ biến cho dataset spam

    # Đổi tên cột cho nhất quán nếu cần
    df = df.rename(columns={df.columns[0]: label_col, df.columns[1]: message_col})
    df = df[[label_col, message_col]].dropna()

    print("\nPhân phối nhãn ban đầu:")
    print(df[label_col].value_counts())

    label_counts = Counter(df[label_col])
    major_class_label, major_count = label_counts.most_common(1)[0]
    minor_class_label, minor_count = label_counts.most_common()[-1]

    if major_count == minor_count:
        print("\n✅ Dữ liệu đã cân bằng. Không cần augmentation.")
        return df

    num_to_generate = major_count - minor_count
    minority_messages = df[df[label_col] == minor_class_label][message_col].tolist()

    print(f"\nCần tạo thêm {num_to_generate} mẫu cho lớp '{minor_class_label}'.")

    augmented_results = []

    # Sử dụng ThreadPoolExecutor để chạy song song với thanh tiến trình tqdm
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Tạo danh sách các tác vụ
        futures = [executor.submit(_back_translate_robust, random.choice(minority_messages)) for _ in range(num_to_generate)]

        # Thu thập kết quả khi chúng hoàn thành và hiển thị tiến trình
        for future in tqdm(as_completed(futures), total=num_to_generate, desc="Tổng hợp câu mới"):
            result = future.result()
            if result: # Chỉ thêm vào nếu kết quả không phải None
                augmented_results.append(result)

    print(f"\n✨ Đã tạo thành công {len(augmented_results)} câu mới độc nhất.")

    # Tạo DataFrame từ dữ liệu mới và kết hợp với dữ liệu gốc
    df_new = pd.DataFrame({
        label_col: [minor_class_label] * len(augmented_results),
        message_col: augmented_results
    })

    df_augmented = pd.concat([df, df_new], ignore_index=True)

    # Trộn ngẫu nhiên dữ liệu
    df_augmented = df_augmented.sample(frac=1).reset_index(drop=True)

    return df_augmented

print("✅ Các hàm tối ưu đã được định nghĩa.")

In [ ]:
# Cell 4: Execute Augmentation and Save
# Sử dụng Google Colab để load nhanh dữ liệu

INPUT_FILE_PATH = '/content/data/spam.csv'
OUTPUT_FILE_PATH = '/content/spam_augmented.csv'

# --- Bắt đầu đo thời gian ---
start_time = time.time()

# Gọi hàm chính để thực hiện công việc
df_final = augment_data_parallel(
    input_path=INPUT_FILE_PATH,
    message_col='Message', # Tên cột tin nhắn
    label_col='Category'   # Tên cột nhãn
)

# --- Kết thúc đo thời gian ---
end_time = time.time()
elapsed_time = end_time - start_time

print("\n" + "="*50)
print("🏁 QUÁ TRÌNH HOÀN TẤT 🏁")
print(f"Tổng thời gian thực thi: {elapsed_time:.2f} giây")
print("="*50)

print("\nPhân phối nhãn sau khi augmentation:")
print(df_final['Category'].value_counts())

# Lưu file kết quả
df_final.to_csv(OUTPUT_FILE_PATH, index=False, encoding='utf-8')
print(f"\n✅ Dữ liệu đã được lưu vào file: {OUTPUT_FILE_PATH}")

In [ ]:
# Cell 5: Download the Result
# sử dụng google colab để load nhanh dữ liệu
# from google.colab import files

'''try:
    files.download(OUTPUT_FILE_PATH)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{OUTPUT_FILE_PATH}'. Hãy chắc chắn rằng ô trước đã chạy thành công.")'''